# Import the source data
The data is provided as a directory that is three levels deep (the third level is ommited in the following listing).
``` bash
fiete@ubu:~/Documents/studium/analyse_semi_und_unstrukturierter_daten$ tree -d -L 1 CAPTUM
CAPTUM
├── Allergic Diseases
├── ANA
├── Angioedema
├── anti-FcεRI
├── Antihistamine
├── Anti-IgE
├── anti-TPO IgE ratio
├── ASST
├── Basophil
├── BAT
├── BHRA
├── CRP
├── Cyclosporine
├── D-Dimer
├── dsDNA
├── Duration
├── Eosinophil
├── IL-24
├── Omalizumab
├── Severity
├── Thyroglobulin
├── Total IgE
└── TPO
```

To work further with the source data, it is useful to have a list of file paths for the pdfs. The following creates a list of all pdf files in the `CAPTUM` source folder.

In [15]:
import os

path = './CAPTUM'

pdf_filepaths = []
for root, directories, files in os.walk(path, topdown=False):
	for name in files:
		pdf_filepaths.append(os.path.join(root, name))

pdf_filepaths[:5]

['./CAPTUM/CRP/ANA/Asero 2017.pdf',
 './CAPTUM/CRP/ANA/Magen 2015.pdf',
 './CAPTUM/CRP/Severity/Kolkhir 2017 .pdf',
 './CAPTUM/CRP/Severity/Baek 2014.pdf',
 './CAPTUM/CRP/Severity/Kasperska-Zajac 2015.pdf']

## Check data for duplicate entries
We can identify duplicate pdfs by computing the checksum of each file and then counting the unique values. So let us define the checksum function `get_checksum()`:

In [16]:
# https://stackoverflow.com/questions/16874598/how-do-i-calculate-the-md5-checksum-of-a-file-in-python#16876405
import hashlib

def get_checksum(filepath: str) -> str:
    # Open,close, read file and calculate MD5 on its contents 
    with open(filepath, 'rb') as file_to_check:
        # read contents of the file
        data = file_to_check.read()    
        # pipe contents of the file through
        return hashlib.md5(data).hexdigest()

# check that it works
file_one, file_one_copy, file_two = "./pdf_1.pdf", "./pdf_1 copy.pdf", "./pdf_2.pdf"
assert get_checksum(file_one) == get_checksum(file_one_copy), "should be equal"
assert get_checksum(file_one) != get_checksum(file_two), "should not be equal"

Then we can create a pandas dataframe from the list of filepath's and also add a checksum column that is computed using our `get_checksum()` function.

In [17]:
import pandas as pd
df = pd.DataFrame(pdf_filepaths, columns = ['filepath'])
df['checksum'] = df['filepath'].apply(get_checksum)
df

,filepath,checksum
0,./CAPTUM/CRP/ANA/Asero 2017.pdf,2fad223ae2232cb9e855d3ece9e34b72
1,./CAPTUM/CRP/ANA/Magen 2015.pdf,c721aaea67a47811324b3c860dde612b
2,./CAPTUM/CRP/Severity/Kolkhir 2017 .pdf,aed2cb292fdffefe2a319b9d7e517bb3
3,./CAPTUM/CRP/Severity/Baek 2014.pdf,989e3eca08259c9a898acc551473f55f
4,./CAPTUM/CRP/Severity/Kasperska-Zajac 2015.pdf,2ed156f4fd5cfa00198f3f6f590940e0
...,...,...
1054,./CAPTUM/Omalizumab/Cyclosporine/Rosenblum 202...,fb22292adf8f35656fde0e54dc0cee51
1055,./CAPTUM/Omalizumab/Cyclosporine/Gimenez Arnau...,6a5635468c99716fc18b91b7b6ebaeaf
1056,./CAPTUM/Omalizumab/Cyclosporine/Koski 2017.pdf,6cfd7540663be0f6d7fb72f776339b71
1057,./CAPTUM/Omalizumab/Cyclosporine/Ke 2017.pdf,849adffe6101df0a030cf425f661e1ed


In the final step, we can analyse the results of this activity. It seems that our available data is in reality only half as large as it initially appears.

In [18]:
print('Total number of pdfs: {}'.format(df['checksum'].count()))
print('Total number of unique pdfs: {}'.format(len(df['checksum'].unique())))
df['checksum']


Total number of pdfs: 1059
Total number of unique pdfs: 466


0       2fad223ae2232cb9e855d3ece9e34b72
1       c721aaea67a47811324b3c860dde612b
2       aed2cb292fdffefe2a319b9d7e517bb3
3       989e3eca08259c9a898acc551473f55f
4       2ed156f4fd5cfa00198f3f6f590940e0
                      ...               
1054    fb22292adf8f35656fde0e54dc0cee51
1055    6a5635468c99716fc18b91b7b6ebaeaf
1056    6cfd7540663be0f6d7fb72f776339b71
1057    849adffe6101df0a030cf425f661e1ed
1058    f13be81ffbff55e031a34ef81d43cbff
Name: checksum, Length: 1059, dtype: object

Now we create a df of unique pdfs by removing duplicate checksums

In [19]:
df_unique = df.drop_duplicates(subset=['checksum'])
df_unique

,filepath,checksum
0,./CAPTUM/CRP/ANA/Asero 2017.pdf,2fad223ae2232cb9e855d3ece9e34b72
1,./CAPTUM/CRP/ANA/Magen 2015.pdf,c721aaea67a47811324b3c860dde612b
2,./CAPTUM/CRP/Severity/Kolkhir 2017 .pdf,aed2cb292fdffefe2a319b9d7e517bb3
3,./CAPTUM/CRP/Severity/Baek 2014.pdf,989e3eca08259c9a898acc551473f55f
4,./CAPTUM/CRP/Severity/Kasperska-Zajac 2015.pdf,2ed156f4fd5cfa00198f3f6f590940e0
...,...,...
1050,./CAPTUM/Omalizumab/Cyclosporine/Sánchez 2020.pdf,167014bf6002af9c8c62794730d3473e
1053,./CAPTUM/Omalizumab/Cyclosporine/Seth 2016.pdf,0bd4ab01b7e3f79ea404da26da3834b1
1054,./CAPTUM/Omalizumab/Cyclosporine/Rosenblum 202...,fb22292adf8f35656fde0e54dc0cee51
1056,./CAPTUM/Omalizumab/Cyclosporine/Koski 2017.pdf,6cfd7540663be0f6d7fb72f776339b71


The next step is to read the text from the pdfs.   
First we import the neccessary modules (more on pdfminer.six [here](https://pdfminersix.readthedocs.io/en/latest/)) and create a procedure to extract the text.

In [20]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

def convert_pdf_to_string(file_path):
	output_string = StringIO()
	with open(file_path, 'rb') as in_file:
	    parser = PDFParser(in_file)
	    doc = PDFDocument(parser)
	    rsrcmgr = PDFResourceManager()
	    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
	    interpreter = PDFPageInterpreter(rsrcmgr, device)
	    for page in PDFPage.create_pages(doc):
	        interpreter.process_page(page)

	return(output_string.getvalue())

In [21]:
# this takes between five and ten minutes
df_unique['text'] = df_unique.filepath.apply(lambda fp: convert_pdf_to_string(fp))
df_unique.to_csv('captum.csv')

<ipython-input-21-4f1570f4a70c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['text'] = df_unique.filepath.apply(lambda fp: convert_pdf_to_string(fp))


Now we iterate over each individual file and use pdfminer to get the file content.

In [9]:
# Remove Output Path if it exists, then create new
import shutil
out_dir = "./out/"
if os.path.exists(out_dir):
    shutil.rmtree(out_dir)
os.makedirs(out_dir)

# Iterate over each individual file
for index, row in df_unique.iterrows():
    path = row['filepath']
    checksum = row['checksum']

    content = convert_pdf_to_string(path)
    
    out_file_path = os.path.join(out_dir, checksum + ".txt")
    
    with open(out_file_path, "w", encoding="utf-8") as text_file:
        text_file.write(content)
        
    if ((index + 1) % 10 == 0):
        print(str(index + 1) + "/" + str(len(df_unique)) + " done")

10/466 done
20/466 done
30/466 done
50/466 done
70/466 done
80/466 done
90/466 done
100/466 done
110/466 done
130/466 done
140/466 done
150/466 done
160/466 done
170/466 done
180/466 done
190/466 done
200/466 done
220/466 done
230/466 done
310/466 done
330/466 done
340/466 done
350/466 done
360/466 done
370/466 done
400/466 done
450/466 done
480/466 done
500/466 done
510/466 done
520/466 done
560/466 done
580/466 done
590/466 done
600/466 done
630/466 done
640/466 done
660/466 done
720/466 done
730/466 done
750/466 done
760/466 done
780/466 done
790/466 done
860/466 done
900/466 done
930/466 done
950/466 done
970/466 done
